Raccoon vs. Pandas speed test
=============================

Setup pythonpath, import libraries and initialized DataFrame to store results

In [16]:
# Use this statement to import the current development version
import sys; sys.path.insert(0, '../')

In [17]:
from copy import deepcopy

In [18]:
import raccoon as rc
import pandas as pd

Machine information
--------

In [21]:
import platform
print(platform.machine())
print(platform.processor())
print(platform.platform())
print("python ", platform.python_version())

AMD64
Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
Windows-10-10.0.18362-SP0
python  3.7.4


Run the Speed Test
----

In [22]:
results = rc.DataFrame(columns=['raccoon', 'pandas', 'ratio'], sort=False)

In [23]:
def add_results(index):
    results[index, 'raccoon'] = res_rc.best
    results[index, 'pandas'] = res_pd.best
    results[index, 'ratio'] = res_rc.best / res_pd.best

In [24]:
results['version', 'raccoon'] = rc.__version__
results['version', 'pandas'] = pd.__version__
print(results)

index    raccoon    pandas    ratio
-------  ---------  --------  -------
version  3.0.0      0.25.2


Initialize 10,000 empty DataFrames
---------------------------

In [25]:
def init_rc():
    for x in range(10000):
        df = rc.DataFrame()
        
def init_pd():
    for x in range(10000):
        df = pd.DataFrame()

In [26]:
res_rc = %timeit -o init_rc()

89.5 ms ± 6.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
res_pd = %timeit -o init_pd()

4.34 s ± 87.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
add_results('initialize empty')

In [29]:
results.print()

index             raccoon              pandas                 ratio
----------------  -------------------  -----------------  ---------
version           3.0.0                0.25.2
initialize empty  0.08051184000000262  4.216560099999981  0.0190942


Initialize 100 row X 100 col DataFrame()
--------

In [30]:
data = dict()
for x in range(100):
    data['a' + str(x)] = list(range(100))

In [31]:
res_rc = %timeit -o df=rc.DataFrame(data=data, sort=False)

117 µs ± 6.99 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
res_pd = %timeit -o df=pd.DataFrame(data=data)

16 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
add_results('initialize with matrix')

In [34]:
results.print()

index                   raccoon                 pandas                     ratio
----------------------  ----------------------  --------------------  ----------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981     0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276  0.00724737


Add 10,000 items in 1 column to empty DataFrame
-------------

In [35]:
def one_col_add_rc():
    df = rc.DataFrame()
    for x in range(10000):
        df.set(x, 'a', x)
        
def one_col_add_pd():
    df = pd.DataFrame()
    for x in range(10000):
        df.at[x, 'a'] = x

In [36]:
res_rc = %timeit -o one_col_add_rc()

53.3 ms ± 877 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
res_pd = %timeit -o one_col_add_pd()

17 s ± 221 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
add_results('add rows one column')

In [39]:
print(results)

index                   raccoon                 pandas                     ratio
----------------------  ----------------------  --------------------  ----------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981     0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276  0.00724737
add rows one column     0.05175027999999884     16.65805690000002     0.00310662


Add 100 rows of 100 columns to empty DataFrame
----------

In [40]:
new_row = {('a' + str(x)): x for x in range(100)}
columns = ['a' + str(x) for x in range(100)]

def matrix_add_rc():
    df = rc.DataFrame(columns=columns)
    for x in range(100):
        df.set(indexes=x, values=new_row)

def matrix_add_pd():
    df = pd.DataFrame(columns=columns)
    for x in range(100):
        df.loc[x] = new_row

In [41]:
res_rc = %timeit -o matrix_add_rc()

10.1 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
res_pd = %timeit -o matrix_add_pd()

315 ms ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
add_results('add matrix')

In [44]:
print(results)

index                   raccoon                 pandas                     ratio
----------------------  ----------------------  --------------------  ----------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981     0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276  0.00724737
add rows one column     0.05175027999999884     16.65805690000002     0.00310662
add matrix              0.009487011999999595    0.29534590000002936   0.0321217


Append 10x10 DataFrame 1000 times
------

In [45]:
def append_rc():
    grid = {'a' + str(x): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] for x in range(10)}
    df = rc.DataFrame(data=deepcopy(grid), columns=list(grid.keys()))
    for x in range(100):
        index = [(y + 1) + (x + 1) * 10 for y in range(10)]
        new_grid = deepcopy(grid)
        new_df = rc.DataFrame(data=new_grid, columns=list(new_grid.keys()), index=index)
        df.append(new_df)

def append_pd():
    grid = {'a' + str(x): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] for x in range(10)}
    df = pd.DataFrame(data=grid, columns=list(grid.keys()))
    for x in range(100):
        index = [(y + 1) + (x + 1) * 10 for y in range(10)]
        new_grid = deepcopy(grid)
        new_df = pd.DataFrame(data=new_grid, columns=list(new_grid.keys()), index=index)
        df = df.append(new_df)

In [46]:
res_rc = %timeit -o append_rc()

77.4 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
res_pd = %timeit -o append_pd()

368 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
add_results('append')

In [49]:
print(results)

index                   raccoon                 pandas                     ratio
----------------------  ----------------------  --------------------  ----------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981     0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276  0.00724737
add rows one column     0.05175027999999884     16.65805690000002     0.00310662
add matrix              0.009487011999999595    0.29534590000002936   0.0321217
append                  0.07532331999999542     0.35071719999996276   0.214769


Get
---

In [50]:
# First create a 1000 row X 100 col matrix for the test. Index is [0...999]

col = [x for x in range(1000)]
grid = {'a' + str(x): col[:] for x in range(100)}

df_rc = rc.DataFrame(data=grid, columns=sorted(grid.keys()))
df_pd = pd.DataFrame(data=grid, columns=sorted(grid.keys()))

In [51]:
# get cell

def rc_get_cell():
    for c in df_rc.columns:
        for r in df_rc.index:
            x = df_rc.get(r, c)
            
def pd_get_cell():
    for c in df_pd.columns:
        for r in df_pd.index:
            x = df_pd.at[r, c]

In [52]:
res_rc = %timeit -o rc_get_cell()

746 ms ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
res_pd = %timeit -o pd_get_cell()

1.03 s ± 32.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
add_results('get cell')

In [55]:
print(results)

index                   raccoon                 pandas                     ratio
----------------------  ----------------------  --------------------  ----------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981     0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276  0.00724737
add rows one column     0.05175027999999884     16.65805690000002     0.00310662
add matrix              0.009487011999999595    0.29534590000002936   0.0321217
append                  0.07532331999999542     0.35071719999996276   0.214769
get cell                0.6899712000000591      0.9961774999999307    0.692619


In [56]:
# get column all index

def get_column_all_rc():
    for c in df_rc.columns:
        x = df_rc.get(columns=c)
        
def get_column_all_pd():
    for c in df_pd.columns:
        x = df_pd[c]

In [57]:
res_rc = %timeit -o get_column_all_rc()

46.8 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
res_pd = %timeit -o get_column_all_pd()

423 µs ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [59]:
add_results('get column all index')

In [60]:
print(results)

index                   raccoon                 pandas                         ratio
----------------------  ----------------------  ----------------------  ------------
version                 3.0.0                   0.25.2
initialize empty        0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix  0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column     0.05175027999999884     16.65805690000002         0.00310662
add matrix              0.009487011999999595    0.29534590000002936       0.0321217
append                  0.07532331999999542     0.35071719999996276       0.214769
get cell                0.6899712000000591      0.9961774999999307        0.692619
get column all index    0.04172945999999911     0.00037849979999998593  110.25


In [61]:
# get subset of the index of the column

def get_column_subset_rc():
    for c in df_rc.columns:
        for r in range(100):
            rows = list(range(r*10, r*10 + 9))
            x = df_rc.get(indexes=rows, columns=c)
        
def get_column_subset_pd():
    for c in df_pd.columns:
        for r in range(100):
            rows = list(range(r*10, r*10 + 9))
            x = df_pd.loc[rows, c]

In [62]:
res_rc = %timeit -o get_column_subset_rc()

637 ms ± 51.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [63]:
res_pd = %timeit -o get_column_subset_pd()

6.91 s ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
add_results('get column subset index')

In [65]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779


In [66]:
# get index all columns

def get_index_all_rc():
    for i in df_rc.index:
        x = df_rc.get(indexes=i)
        
def get_index_all_pd():
    for i in df_pd.index:
        x = df_pd.loc[i]

In [67]:
res_rc = %timeit -o get_index_all_rc()

1.09 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [68]:
res_pd = %timeit -o get_index_all_pd()

240 ms ± 18.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
add_results('get index all columns')

In [70]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459


Set
---

In [71]:
# First create a 1000 row X 100 col matrix for the test. Index is [0...999]

col = [x for x in range(1000)]
grid = {'a' + str(x): col[:] for x in range(100)}

df_rc = rc.DataFrame(data=grid, columns=sorted(grid.keys()))
df_pd = pd.DataFrame(data=grid, columns=sorted(grid.keys()))

In [72]:
# set cell

def rc_set_cell():
    for c in df_rc.columns:
        for r in df_rc.index:
            df_rc.set(r, c, 99)
            
def pd_set_cell():
    for c in df_pd.columns:
        for r in df_pd.index:
            df_pd.at[r, c] = 99

In [73]:
res_rc = %timeit -o rc_set_cell()

593 ms ± 64.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
res_pd = %timeit -o pd_set_cell()

1.35 s ± 62.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [75]:
add_results('set cell')

In [76]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

In [77]:
# set column all index

def set_column_all_rc():
    for c in df_rc.columns:
        x = df_rc.set(columns=c, values=99)
        
def set_column_all_pd():
    for c in df_pd.columns:
        x = df_pd[c] = 99

In [78]:
res_rc = %timeit -o set_column_all_rc()

6.03 ms ± 761 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
res_pd = %timeit -o set_column_all_pd()

16.6 ms ± 521 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
add_results('set column all index')

In [81]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

In [82]:
# set subset of the index of the column

def set_column_subset_rc():
    for c in df_rc.columns:
        for r in range(100):
            rows = list(range(r*10, r*10 + 10))
            x = df_rc.set(indexes=rows, columns=c, values=list(range(10)))
        
def set_column_subset_pd():
    for c in df_pd.columns:
        for r in range(100):
            rows = list(range(r*10, r*10 + 10))
            x = df_pd.loc[rows, c] = list(range(10))

In [83]:
res_rc = %timeit -o set_column_subset_rc()

413 ms ± 60.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
res_pd = %timeit -o set_column_subset_pd()

1min ± 660 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
add_results('set column subset index')

In [86]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

In [87]:
row = {x:x for x in grid.keys()}

In [88]:
# set index all columns

def set_index_all_rc():
    for i in df_rc.index:
        x = df_rc.set(indexes=i, values=row)
        
def set_index_all_pd():
    for i in df_pd.index:
        x = df_pd.loc[i] = row

In [89]:
res_rc = %timeit -o set_index_all_rc()

65.5 ms ± 7.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [90]:
res_pd = %timeit -o set_index_all_pd()

1.36 s ± 63.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
add_results('set index all columns')

In [92]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

Sort
-----

In [93]:
# make a dataframe 1000x100 with index in reverse order

rev = list(reversed(range(1000)))

df_rc = rc.DataFrame(data=grid, index=rev)
df_pd = pd.DataFrame(grid, index=rev)

In [94]:
res_rc = %timeit -o df_rc.sort_index() 

12.1 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
res_pd = %timeit -o df_pd.sort_index()

369 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [96]:
add_results('sort index')

In [97]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

Iterators
---------

In [98]:
# First create a 1000 row X 100 col matrix for the test. Index is [0...999]

col = [x for x in range(1000)]
grid = {'a' + str(x): col[:] for x in range(100)}

df_rc = rc.DataFrame(data=grid, columns=sorted(grid.keys()))
df_pd = pd.DataFrame(data=grid, columns=sorted(grid.keys()))

In [99]:
# iterate over the rows

def iter_rc():
    for row in df_rc.iterrows():
        x = row
        
def iter_pd():
    for row in df_pd.itertuples():
        x = row

In [100]:
res_rc = %timeit -o iter_rc() 

The slowest run took 1823.08 times longer than the fastest. This could mean that an intermediate result is being cached.
3.06 s ± 7.45 s per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [101]:
res_pd = %timeit -o iter_pd()

27.3 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
add_results('iterate rows')

In [103]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

Insert in the middle
--------------------

In [104]:
# First create a 500 row X 100 col matrix for the test. Index is [1, 3, 5, 7,...500] every other

col = [x for x in range(1, 1000, 2)]
grid = {'a' + str(x): col[:] for x in range(100)}

df_rc = rc.DataFrame(data=grid, columns=sorted(grid.keys()), sort=True)
df_pd = pd.DataFrame(data=grid, columns=sorted(grid.keys()))

In [105]:
row = {x:x for x in grid.keys()}

In [106]:
# set index all columns

def insert_rows_rc():
    for i in range(0, 999, 2):
        x = df_rc.set(indexes=i, values=row)
        
def insert_rows_pd():
    for i in range(0, 999, 2):
        x = df_pd.loc[i] = row

In [107]:
res_rc = %timeit -o insert_rows_rc() 

19.3 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
res_pd = %timeit -o insert_rows_pd()

300 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [109]:
add_results('insert rows')

In [110]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      

Time Series Append
------------------
Simulate the recording of a stock on 1 minute intervals and appending to the DataFrame

In [113]:
data_row = {'open': 100, 'high': 101, 'low': 99, 'close': 100.5, 'volume': 999}

dates = pd.date_range('2010-01-01 09:30:00', periods=10000, freq='1min')

def time_series_rc():
    ts = rc.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'], index_name='datetime', sort=True)
    for date in dates:
        ts.set_row(date, data_row)

def time_series_pd():
    ts = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
    for date in dates:
        ts.loc[date] = data_row

In [114]:
res_rc = %timeit -o time_series_rc() 

54.9 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
res_pd = %timeit -o time_series_pd()

12.9 s ± 707 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [116]:
add_results('time series')

In [117]:
print(results)

index                    raccoon                 pandas                         ratio
-----------------------  ----------------------  ----------------------  ------------
version                  3.0.0                   0.25.2
initialize empty         0.08051184000000262     4.216560099999981         0.0190942
initialize with matrix   0.00011038290999999845  0.015230743999999276      0.00724737
add rows one column      0.05175027999999884     16.65805690000002         0.00310662
add matrix               0.009487011999999595    0.29534590000002936       0.0321217
append                   0.07532331999999542     0.35071719999996276       0.214769
get cell                 0.6899712000000591      0.9961774999999307        0.692619
get column all index     0.04172945999999911     0.00037849979999998593  110.25
get column subset index  0.5775211000000127      6.717084499999942         0.0859779
get index all columns    1.0432272999998986      0.21270430000004126       4.90459
set cell      